In [13]:
import numpy as np
from obspy.core import UTCDateTime
from obspy import read
from obspy.taup import TauPyModel
from obspy.geodetics import locations2degrees
from glob import glob
import os
import os.path

In [23]:
project_name = 'june30'
GF_list = 'rc_gflist.gflist' # Station file used in fq gen code
vel_mod = 'mojave'
stas_name = GF_list.split('.')[0]
fq_dir = '/hdd/rc_fq/summer23/' + project_name + '/'

ruptures = np.load('/hdd/rc_fq/summer23/' + project_name + '_ruptures.npy')
test_ruptures = ['june30.000000', 'june30.000001', 'june30.000002']

# Where to save arrival times

arrival_save_dir = '/hdd/rc_fq/summer23/' + project_name + '_fq_parrivals/'
if os.path.isdir(arrival_save_dir):
    pass
else:
    os.makedirs(arrival_save_dir)

In [46]:
# w = open('/home/sdybing/rc_fq/test_p_arrivals.csv', 'w') # saves arrivals for every event in one csv. Not using currently
counter = 0

for rupture in ruptures:
    
    counter += 1
    w = open(arrival_save_dir + stas_name + '_arrival_times_' + rupture + '.csv', 'w') # saves arrivals for each event in a separate csv
    
    print('Rupture ' + rupture + ' (' + str(counter) + '/' + str(len(ruptures)) + ')')
    
    log = glob(fq_dir + 'output/ruptures/' + rupture + '.log')
    f = open(log[0],'r')
    line = f.readlines()
    
    # getting hypocenter location
    hyp_loc_junk = line[16]
    hyp_loc_1 = float(hyp_loc_junk.split(' ')[2].split('(')[1].split(')')[0].split(',')[0])
    hyp_loc_2 = float(hyp_loc_junk.split(' ')[2].split('(')[1].split(')')[0].split(',')[1])
    hyp_loc_3 = float(hyp_loc_junk.split(' ')[2].split('(')[1].split(')')[0].split(',')[2])
    
    epicenter = []
    epicenter.append(hyp_loc_1)
    epicenter.append(hyp_loc_2)
    epicenter.append(hyp_loc_3)
#     print(epicenter)
    
    epi_lon = epicenter[0]
    epi_lat = epicenter[1]
    epi_depth = epicenter[2]
    
    # getting hypocenter time
    hyp_time_junk = line[17]
    hyp_time = hyp_time_junk.split(' ')[2].split('Z')[0]
    time_epi = UTCDateTime(hyp_time)  
    stations = np.load('/hdd/rc_fq/summer23/' + stas_name + '_station_names.npy')
    Nsta = len(stations) # number of stations
    
    rootpath = fq_dir + '/output/waveforms/' + rupture + '/' # waveform file locations
    lonlat_load = np.load('/hdd/rc_fq/summer23/' + stas_name + '_station_namesandlocs.npy') 
    lonlat = lonlat_load[:,0:2] # station locations
    
    station_catalogue_load = np.load('/hdd/rc_fq/summer23/' + stas_name + '_station_namesandlocs.npy')
    station_catalogue = station_catalogue_load[:,2]
    
    velmod = TauPyModel(model = fq_dir + '/structure/' + vel_mod + '.npz') # velocity model
    
    predictedP = 9999 * np.ones(len(stations))
    
    # Get predicted arrivals
    
    for ksta in range(len(stations)):
        
        # Find station coordinates
        i = np.where(station_catalogue == stations[ksta])[0]
        lon_sta = float(lonlat[i,0])
        lat_sta = float(lonlat[i,1])
                        
        deg = locations2degrees(lat_sta, lon_sta, epi_lat, epi_lon) # calculates distance between station loc and epicenter loc
        arrivals = velmod.get_travel_times(source_depth_in_km = epi_depth, distance_in_degree = deg, phase_list = ['P','Pn','p'])
        
        # Determine P and S arrivals
        for kphase in range(len(arrivals)):
            if 'P' == arrivals[kphase].name or 'p' == arrivals[kphase].name or 'Pn' == arrivals[kphase].name:
                if arrivals[kphase].time < predictedP[ksta]:
                    predictedP[ksta] = arrivals[kphase].time
                    
#         print('Station ' + stations[ksta] + ': predicted arrival = ' + str(time_epi + predictedP[ksta]))
        
        line = '%s\t%s\t%s\n'%(rupture, str(stations[ksta]), str(time_epi + predictedP[ksta]))
        # print(line)
        w.write(line)
    
    w.close()

# w.close()



Rupture june30.000000 (1/3300)
Rupture june30.000001 (2/3300)
Rupture june30.000002 (3/3300)
Rupture june30.000003 (4/3300)
Rupture june30.000004 (5/3300)
Rupture june30.000005 (6/3300)
Rupture june30.000006 (7/3300)
Rupture june30.000007 (8/3300)
Rupture june30.000008 (9/3300)
Rupture june30.000009 (10/3300)
Rupture june30.000010 (11/3300)
Rupture june30.000011 (12/3300)
Rupture june30.000012 (13/3300)
Rupture june30.000013 (14/3300)
Rupture june30.000014 (15/3300)
Rupture june30.000015 (16/3300)
Rupture june30.000016 (17/3300)
Rupture june30.000017 (18/3300)
Rupture june30.000018 (19/3300)
Rupture june30.000019 (20/3300)
Rupture june30.000020 (21/3300)
Rupture june30.000021 (22/3300)
Rupture june30.000022 (23/3300)
Rupture june30.000023 (24/3300)
Rupture june30.000024 (25/3300)
Rupture june30.000025 (26/3300)
Rupture june30.000026 (27/3300)
Rupture june30.000027 (28/3300)
Rupture june30.000028 (29/3300)
Rupture june30.000029 (30/3300)
Rupture june30.000030 (31/3300)
Rupture june30.00

Rupture june30.000252 (253/3300)
Rupture june30.000253 (254/3300)
Rupture june30.000254 (255/3300)
Rupture june30.000255 (256/3300)
Rupture june30.000256 (257/3300)
Rupture june30.000257 (258/3300)
Rupture june30.000258 (259/3300)
Rupture june30.000259 (260/3300)
Rupture june30.000260 (261/3300)
Rupture june30.000261 (262/3300)
Rupture june30.000262 (263/3300)
Rupture june30.000263 (264/3300)
Rupture june30.000264 (265/3300)
Rupture june30.000265 (266/3300)
Rupture june30.000266 (267/3300)
Rupture june30.000267 (268/3300)
Rupture june30.000268 (269/3300)
Rupture june30.000269 (270/3300)
Rupture june30.000270 (271/3300)
Rupture june30.000271 (272/3300)
Rupture june30.000272 (273/3300)
Rupture june30.000273 (274/3300)
Rupture june30.000274 (275/3300)
Rupture june30.000275 (276/3300)
Rupture june30.000276 (277/3300)
Rupture june30.000277 (278/3300)
Rupture june30.000278 (279/3300)
Rupture june30.000279 (280/3300)
Rupture june30.000280 (281/3300)
Rupture june30.000281 (282/3300)
Rupture ju

Rupture june30.000501 (502/3300)
Rupture june30.000502 (503/3300)
Rupture june30.000503 (504/3300)
Rupture june30.000504 (505/3300)
Rupture june30.000505 (506/3300)
Rupture june30.000506 (507/3300)
Rupture june30.000507 (508/3300)
Rupture june30.000508 (509/3300)
Rupture june30.000509 (510/3300)
Rupture june30.000510 (511/3300)
Rupture june30.000511 (512/3300)
Rupture june30.000512 (513/3300)
Rupture june30.000513 (514/3300)
Rupture june30.000514 (515/3300)
Rupture june30.000515 (516/3300)
Rupture june30.000516 (517/3300)
Rupture june30.000517 (518/3300)
Rupture june30.000518 (519/3300)
Rupture june30.000519 (520/3300)
Rupture june30.000520 (521/3300)
Rupture june30.000521 (522/3300)
Rupture june30.000522 (523/3300)
Rupture june30.000523 (524/3300)
Rupture june30.000524 (525/3300)
Rupture june30.000525 (526/3300)
Rupture june30.000526 (527/3300)
Rupture june30.000527 (528/3300)
Rupture june30.000528 (529/3300)
Rupture june30.000529 (530/3300)
Rupture june30.000530 (531/3300)
Rupture ju

Rupture june30.000750 (751/3300)
Rupture june30.000751 (752/3300)
Rupture june30.000752 (753/3300)
Rupture june30.000753 (754/3300)
Rupture june30.000754 (755/3300)
Rupture june30.000755 (756/3300)
Rupture june30.000756 (757/3300)
Rupture june30.000757 (758/3300)
Rupture june30.000758 (759/3300)
Rupture june30.000759 (760/3300)
Rupture june30.000760 (761/3300)
Rupture june30.000761 (762/3300)
Rupture june30.000762 (763/3300)
Rupture june30.000763 (764/3300)
Rupture june30.000764 (765/3300)
Rupture june30.000765 (766/3300)
Rupture june30.000766 (767/3300)
Rupture june30.000767 (768/3300)
Rupture june30.000768 (769/3300)
Rupture june30.000769 (770/3300)
Rupture june30.000770 (771/3300)
Rupture june30.000771 (772/3300)
Rupture june30.000772 (773/3300)
Rupture june30.000773 (774/3300)
Rupture june30.000774 (775/3300)
Rupture june30.000775 (776/3300)
Rupture june30.000776 (777/3300)
Rupture june30.000777 (778/3300)
Rupture june30.000778 (779/3300)
Rupture june30.000779 (780/3300)
Rupture ju

Rupture june30.000999 (1000/3300)
Rupture june30.001000 (1001/3300)
Rupture june30.001001 (1002/3300)
Rupture june30.001002 (1003/3300)
Rupture june30.001003 (1004/3300)
Rupture june30.001004 (1005/3300)
Rupture june30.001005 (1006/3300)
Rupture june30.001006 (1007/3300)
Rupture june30.001007 (1008/3300)
Rupture june30.001008 (1009/3300)
Rupture june30.001009 (1010/3300)
Rupture june30.001010 (1011/3300)
Rupture june30.001011 (1012/3300)
Rupture june30.001012 (1013/3300)
Rupture june30.001013 (1014/3300)
Rupture june30.001014 (1015/3300)
Rupture june30.001015 (1016/3300)
Rupture june30.001016 (1017/3300)
Rupture june30.001017 (1018/3300)
Rupture june30.001018 (1019/3300)
Rupture june30.001019 (1020/3300)
Rupture june30.001020 (1021/3300)
Rupture june30.001021 (1022/3300)
Rupture june30.001022 (1023/3300)
Rupture june30.001023 (1024/3300)
Rupture june30.001024 (1025/3300)
Rupture june30.001025 (1026/3300)
Rupture june30.001026 (1027/3300)
Rupture june30.001027 (1028/3300)
Rupture june30

...

Rupture june30.002204 (2205/3300)
Rupture june30.002205 (2206/3300)
Rupture june30.002206 (2207/3300)
Rupture june30.002207 (2208/3300)
Rupture june30.002208 (2209/3300)
Rupture june30.002209 (2210/3300)
Rupture june30.002210 (2211/3300)
Rupture june30.002211 (2212/3300)
Rupture june30.002212 (2213/3300)
Rupture june30.002213 (2214/3300)
Rupture june30.002214 (2215/3300)
Rupture june30.002215 (2216/3300)
Rupture june30.002216 (2217/3300)
Rupture june30.002217 (2218/3300)
Rupture june30.002218 (2219/3300)
Rupture june30.002219 (2220/3300)
Rupture june30.002220 (2221/3300)
Rupture june30.002221 (2222/3300)
Rupture june30.002222 (2223/3300)
Rupture june30.002223 (2224/3300)
Rupture june30.002224 (2225/3300)
Rupture june30.002225 (2226/3300)
Rupture june30.002226 (2227/3300)
Rupture june30.002227 (2228/3300)
Rupture june30.002228 (2229/3300)
Rupture june30.002229 (2230/3300)
Rupture june30.002230 (2231/3300)
Rupture june30.002231 (2232/3300)
Rupture june30.002232 (2233/3300)
Rupture june30

Rupture june30.002445 (2446/3300)
Rupture june30.002446 (2447/3300)
Rupture june30.002447 (2448/3300)
Rupture june30.002448 (2449/3300)
Rupture june30.002449 (2450/3300)
Rupture june30.002450 (2451/3300)
Rupture june30.002451 (2452/3300)
Rupture june30.002452 (2453/3300)
Rupture june30.002453 (2454/3300)
Rupture june30.002454 (2455/3300)
Rupture june30.002455 (2456/3300)
Rupture june30.002456 (2457/3300)
Rupture june30.002457 (2458/3300)
Rupture june30.002458 (2459/3300)
Rupture june30.002459 (2460/3300)
Rupture june30.002460 (2461/3300)
Rupture june30.002461 (2462/3300)
Rupture june30.002462 (2463/3300)
Rupture june30.002463 (2464/3300)
Rupture june30.002464 (2465/3300)
Rupture june30.002465 (2466/3300)
Rupture june30.002466 (2467/3300)
Rupture june30.002467 (2468/3300)
Rupture june30.002468 (2469/3300)
Rupture june30.002469 (2470/3300)
Rupture june30.002470 (2471/3300)
Rupture june30.002471 (2472/3300)
Rupture june30.002472 (2473/3300)
Rupture june30.002473 (2474/3300)
Rupture june30

Rupture june30.002686 (2687/3300)
Rupture june30.002687 (2688/3300)
Rupture june30.002688 (2689/3300)
Rupture june30.002689 (2690/3300)
Rupture june30.002690 (2691/3300)
Rupture june30.002691 (2692/3300)
Rupture june30.002692 (2693/3300)
Rupture june30.002693 (2694/3300)
Rupture june30.002694 (2695/3300)
Rupture june30.002695 (2696/3300)
Rupture june30.002696 (2697/3300)
Rupture june30.002697 (2698/3300)
Rupture june30.002698 (2699/3300)
Rupture june30.002699 (2700/3300)
Rupture june30.002700 (2701/3300)
Rupture june30.002701 (2702/3300)
Rupture june30.002702 (2703/3300)
Rupture june30.002703 (2704/3300)
Rupture june30.002704 (2705/3300)
Rupture june30.002705 (2706/3300)
Rupture june30.002706 (2707/3300)
Rupture june30.002707 (2708/3300)
Rupture june30.002708 (2709/3300)
Rupture june30.002709 (2710/3300)
Rupture june30.002710 (2711/3300)
Rupture june30.002711 (2712/3300)
Rupture june30.002712 (2713/3300)
Rupture june30.002713 (2714/3300)
Rupture june30.002714 (2715/3300)
Rupture june30

Rupture june30.002927 (2928/3300)
Rupture june30.002928 (2929/3300)
Rupture june30.002929 (2930/3300)
Rupture june30.002930 (2931/3300)
Rupture june30.002931 (2932/3300)
Rupture june30.002932 (2933/3300)
Rupture june30.002933 (2934/3300)
Rupture june30.002934 (2935/3300)
Rupture june30.002935 (2936/3300)
Rupture june30.002936 (2937/3300)
Rupture june30.002937 (2938/3300)
Rupture june30.002938 (2939/3300)
Rupture june30.002939 (2940/3300)
Rupture june30.002940 (2941/3300)
Rupture june30.002941 (2942/3300)
Rupture june30.002942 (2943/3300)
Rupture june30.002943 (2944/3300)
Rupture june30.002944 (2945/3300)
Rupture june30.002945 (2946/3300)
Rupture june30.002946 (2947/3300)
Rupture june30.002947 (2948/3300)
Rupture june30.002948 (2949/3300)
Rupture june30.002949 (2950/3300)
Rupture june30.002950 (2951/3300)
Rupture june30.002951 (2952/3300)
Rupture june30.002952 (2953/3300)
Rupture june30.002953 (2954/3300)
Rupture june30.002954 (2955/3300)
Rupture june30.002955 (2956/3300)
Rupture june30

Rupture june30.003168 (3169/3300)
Rupture june30.003169 (3170/3300)
Rupture june30.003170 (3171/3300)
Rupture june30.003171 (3172/3300)
Rupture june30.003172 (3173/3300)
Rupture june30.003173 (3174/3300)
Rupture june30.003174 (3175/3300)
Rupture june30.003175 (3176/3300)
Rupture june30.003176 (3177/3300)
Rupture june30.003177 (3178/3300)
Rupture june30.003178 (3179/3300)
Rupture june30.003179 (3180/3300)
Rupture june30.003180 (3181/3300)
Rupture june30.003181 (3182/3300)
Rupture june30.003182 (3183/3300)
Rupture june30.003183 (3184/3300)
Rupture june30.003184 (3185/3300)
Rupture june30.003185 (3186/3300)
Rupture june30.003186 (3187/3300)
Rupture june30.003187 (3188/3300)
Rupture june30.003188 (3189/3300)
Rupture june30.003189 (3190/3300)
Rupture june30.003190 (3191/3300)
Rupture june30.003191 (3192/3300)
Rupture june30.003192 (3193/3300)
Rupture june30.003193 (3194/3300)
Rupture june30.003194 (3195/3300)
Rupture june30.003195 (3196/3300)
Rupture june30.003196 (3197/3300)
Rupture june30